### Project setup

In [ ]:
!pip install wandb

In [23]:
!wandb login
from wandb.keras import WandbCallback
import wandb
wandb.init(project="slc0")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chandu_ (c01). Use `wandb login --relogin` to force relogin


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1aa66ed7-782c-7749-b791-c3c61d1c420a)


In [4]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [5]:
from tensorflow.keras import mixed_precision
mixed_precision.global_policy()

<Policy "float32">

In [6]:
mixed_precision.set_global_policy(policy="mixed_float16")
mixed_precision.global_policy()

<Policy "mixed_float16">

### Preparing Data

In [7]:
import zipfile 
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/tensorflowCNN/dataset/archive.zip")
zip_ref.extractall()
zip_ref.close()

In [8]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "asl_alphabet_train/asl_alphabet_train",
    label_mode="categorical", 
    image_size=(200, 200),
    validation_split=0.2,
    subset="training",
    seed=42
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "asl_alphabet_train/asl_alphabet_train",
    label_mode="categorical", 
    image_size=(200, 200),
    validation_split=0.2,
    subset="validation",
    seed=42
)

Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


## Modeling

#### model_0(ANN)

In [10]:
import tensorflow as tf

model_0 = tf.keras.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.LayerNormalization(axis=1),
    tf.keras.layers.Dense(300, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(300, activation=tf.keras.activations.relu), 
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_0.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_0 = model_0.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_0")]
)

Epoch 1/3
2175/2175 [==============================] - 62s 27ms/step - loss: 3.6438 - accuracy: 0.1823 - val_loss: 2.7951 - val_accuracy: 0.1753
Epoch 2/3
2175/2175 [==============================] - 56s 26ms/step - loss: 2.5378 - accuracy: 0.2199 - val_loss: 2.2436 - val_accuracy: 0.2684
Epoch 3/3
2175/2175 [==============================] - 58s 27ms/step - loss: 2.0334 - accuracy: 0.3229 - val_loss: 1.7103 - val_accuracy: 0.3886


### model_1 (adding more layers to model_0)

In [11]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.relu), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_1.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_1 = model_1.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_1")]
)

Epoch 1/3
2175/2175 [==============================] - 104s 47ms/step - loss: 3.2205 - accuracy: 0.2892 - val_loss: 1.8313 - val_accuracy: 0.4095
Epoch 2/3
2175/2175 [==============================] - 105s 48ms/step - loss: 1.3205 - accuracy: 0.5605 - val_loss: 1.0822 - val_accuracy: 0.6317
Epoch 3/3
2175/2175 [==============================] - 103s 47ms/step - loss: 0.9648 - accuracy: 0.6898 - val_loss: 0.8402 - val_accuracy: 0.7282


### model_2(tweaking activation function and optimizer)

#### model_2_1(with tanh activation function and SGD optimizer)

In [12]:
model_2_1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.tanh), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.tanh),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.tanh),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_2_1.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.SGD()
)
history_2_1 = model_2_1.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_2")]
)

Epoch 1/3
2175/2175 [==============================] - 76s 35ms/step - loss: 1.4835 - accuracy: 0.5598 - val_loss: 0.9572 - val_accuracy: 0.7034
Epoch 2/3
2175/2175 [==============================] - 75s 35ms/step - loss: 0.8518 - accuracy: 0.7369 - val_loss: 0.8467 - val_accuracy: 0.7275
Epoch 3/3
2175/2175 [==============================] - 75s 34ms/step - loss: 0.7562 - accuracy: 0.7582 - val_loss: 0.8394 - val_accuracy: 0.7321


#### model_2_2(with tanh activation function and Adam optimizer)

In [13]:
model_2_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.tanh), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.tanh),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.tanh),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_2_2.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_2_2 = model_2_2.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_2_2")]
)

Epoch 1/3
2175/2175 [==============================] - 106s 48ms/step - loss: 2.5164 - accuracy: 0.2391 - val_loss: 2.1398 - val_accuracy: 0.3288
Epoch 2/3
2175/2175 [==============================] - 102s 47ms/step - loss: 2.0051 - accuracy: 0.3650 - val_loss: 1.8985 - val_accuracy: 0.3902
Epoch 3/3
2175/2175 [==============================] - 105s 48ms/step - loss: 1.7434 - accuracy: 0.4321 - val_loss: 1.6425 - val_accuracy: 0.4724


#### model_2_3(with sigmoid activation function and Adam optimizer)

In [14]:
model_2_3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.sigmoid), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.sigmoid),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.sigmoid),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_2_3.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_2_3 = model_2_3.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_2_3")]
)

Epoch 1/3
2175/2175 [==============================] - 105s 48ms/step - loss: 2.4806 - accuracy: 0.2290 - val_loss: 2.0308 - val_accuracy: 0.3309
Epoch 2/3
2175/2175 [==============================] - 104s 48ms/step - loss: 1.7395 - accuracy: 0.4253 - val_loss: 1.4625 - val_accuracy: 0.5000
Epoch 3/3
2175/2175 [==============================] - 104s 48ms/step - loss: 1.3430 - accuracy: 0.5459 - val_loss: 1.1863 - val_accuracy: 0.5995


### model_3(add conv2D layers to model_1)
* without normalization

In [15]:
model_3 = tf.keras.Sequential([
    # tf.keras.layers.Rescaling(1/255.),
    tf.keras.layers.Conv2D(10, 3, 1,
                           activation=tf.keras.activations.relu, 
                           input_shape=(200, 200, 3)), 
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, 1),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.relu), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_3.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_3 = model_3.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_3")]
)

Epoch 1/3
2175/2175 [==============================] - 104s 46ms/step - loss: 3.6921 - accuracy: 0.7242 - val_loss: 0.3754 - val_accuracy: 0.8768
Epoch 2/3
2175/2175 [==============================] - 99s 46ms/step - loss: 0.2827 - accuracy: 0.9057 - val_loss: 0.5054 - val_accuracy: 0.8525
Epoch 3/3
2175/2175 [==============================] - 102s 47ms/step - loss: 0.2063 - accuracy: 0.9340 - val_loss: 0.2305 - val_accuracy: 0.9249


### model_3N (model_3 with normalized input images)

In [16]:
model_3N = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1/255.),
    tf.keras.layers.Conv2D(10, 3, 1,
                           activation=tf.keras.activations.relu, 
                           input_shape=(200, 200, 3)), 
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, 1),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.relu), 
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(500, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_3N.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_3N = model_3N.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_3N")]
)

Epoch 1/3
2175/2175 [==============================] - 103s 47ms/step - loss: 0.8671 - accuracy: 0.7200 - val_loss: 0.2470 - val_accuracy: 0.9074
Epoch 2/3
2175/2175 [==============================] - 102s 47ms/step - loss: 0.1750 - accuracy: 0.9390 - val_loss: 0.1792 - val_accuracy: 0.9373
Epoch 3/3
2175/2175 [==============================] - 100s 46ms/step - loss: 0.1090 - accuracy: 0.9631 - val_loss: 0.1339 - val_accuracy: 0.9550


### model_4 (small tweaks on model_3N)

In [17]:
model_4 = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(10, 3, 1,
                           activation=tf.keras.activations.relu, 
                           input_shape=(200, 200, 3)), 
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(10, 3, 1, 
                           activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.relu), 
    tf.keras.layers.Dense(29), 
    tf.keras.layers.Activation(tf.keras.activations.softmax, dtype=tf.float32)
])
model_4.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_4 = model_4.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_4")]
)

Epoch 1/3
2175/2175 [==============================] - 100s 45ms/step - loss: 0.6740 - accuracy: 0.7922 - val_loss: 0.1517 - val_accuracy: 0.9501
Epoch 2/3
2175/2175 [==============================] - 101s 46ms/step - loss: 0.0896 - accuracy: 0.9704 - val_loss: 0.0843 - val_accuracy: 0.9740
Epoch 3/3
2175/2175 [==============================] - 99s 45ms/step - loss: 0.0583 - accuracy: 0.9816 - val_loss: 0.1004 - val_accuracy: 0.9671


### model_5(completely different from previous models but using transfer learning)

In [18]:
mixed_precision.set_global_policy(policy="float32")

In [19]:
base_model = tf.keras.applications.EfficientNetB0(False)
base_model.trainable=False

inputs = tf.keras.layers.Input(shape=(200, 200, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(29)(x)
outputs = tf.keras.layers.Activation(tf.keras.activations.softmax)(x)
model_5 = tf.keras.Model(inputs, outputs)

model_5.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_5 = model_5.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_5")]
)

Epoch 1/3
2175/2175 [==============================] - 140s 61ms/step - loss: 0.3863 - accuracy: 0.9354 - val_loss: 0.0840 - val_accuracy: 0.9890
Epoch 2/3
2175/2175 [==============================] - 129s 59ms/step - loss: 0.0718 - accuracy: 0.9899 - val_loss: 0.0363 - val_accuracy: 0.9945
Epoch 3/3
2175/2175 [==============================] - 133s 61ms/step - loss: 0.0387 - accuracy: 0.9946 - val_loss: 0.0202 - val_accuracy: 0.9970


### model_6(completely different from previous models but using transfer learning)

In [20]:
base_model = tf.keras.applications.ResNet50V2(False)
base_model.trainable=False


inputs= tf.keras.layers.Input(shape=(200, 200, 3))
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(29)(x)
outputs = tf.keras.layers.Activation(tf.keras.activations.softmax)(x)
model_6 = tf.keras.Model(inputs, outputs)

model_6.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_6 = model_6.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_6")]
)

Epoch 1/3
2175/2175 [==============================] - 201s 91ms/step - loss: 0.2927 - accuracy: 0.9387 - val_loss: 0.0691 - val_accuracy: 0.9878
Epoch 2/3
2175/2175 [==============================] - 197s 90ms/step - loss: 0.0505 - accuracy: 0.9919 - val_loss: 0.0322 - val_accuracy: 0.9940
Epoch 3/3
2175/2175 [==============================] - 196s 90ms/step - loss: 0.0248 - accuracy: 0.9962 - val_loss: 0.0198 - val_accuracy: 0.9954


In [21]:
base_model.trainable=True
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_3 True
1 conv1_pad True
2 conv1_conv True
3 pool1_pad True
4 pool1_pool True
5 conv2_block1_preact_bn True
6 conv2_block1_preact_relu True
7 conv2_block1_1_conv True
8 conv2_block1_1_bn True
9 conv2_block1_1_relu True
10 conv2_block1_2_pad True
11 conv2_block1_2_conv True
12 conv2_block1_2_bn True
13 conv2_block1_2_relu True
14 conv2_block1_0_conv True
15 conv2_block1_3_conv True
16 conv2_block1_out True
17 conv2_block2_preact_bn True
18 conv2_block2_preact_relu True
19 conv2_block2_1_conv True
20 conv2_block2_1_bn True
21 conv2_block2_1_relu True
22 conv2_block2_2_pad True
23 conv2_block2_2_conv True
24 conv2_block2_2_bn True
25 conv2_block2_2_relu True
26 conv2_block2_3_conv True
27 conv2_block2_out True
28 conv2_block3_preact_bn True
29 conv2_block3_preact_relu True
30 conv2_block3_1_conv True
31 conv2_block3_1_bn True
32 conv2_block3_1_relu True
33 conv2_block3_2_pad True
34 conv2_block3_2_conv True
35 conv2_block3_2_bn True
36 conv2_block3_2_relu True
37 max_pooling2d_6 Tr

### model_7(finetuning model_6)

In [22]:
tf.keras.models.save_model(model_6, "model_6")

In [23]:
model_7 = tf.keras.models.load_model("model_6")
print(model_6.evaluate(test_data))
print(model_7.evaluate(test_data))

544/544 [==============================] - 47s 87ms/step - loss: 0.0215 - accuracy: 0.9957
[0.021491598337888718, 0.9956896305084229]
544/544 [==============================] - 48s 86ms/step - loss: 0.0215 - accuracy: 0.9957
[0.021491607651114464, 0.9956896305084229]


In [24]:
base_model.trainable = True

for layer in base_model.layers[:-10]:
  layer.trainable = False

model_7.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
)

In [25]:
history_7 = model_5.fit(
    train_data, 
    epochs=5, 
    steps_per_epoch=len(train_data), 
    initial_epoch=2,
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_7")]
)

Epoch 3/5
2175/2175 [==============================] - 133s 61ms/step - loss: 0.0257 - accuracy: 0.9961 - val_loss: 0.0141 - val_accuracy: 0.9977
Epoch 4/5
2175/2175 [==============================] - 133s 61ms/step - loss: 0.0185 - accuracy: 0.9976 - val_loss: 0.0108 - val_accuracy: 0.9977
Epoch 5/5
2175/2175 [==============================] - 133s 61ms/step - loss: 0.0148 - accuracy: 0.9976 - val_loss: 0.0072 - val_accuracy: 0.9982


### model_8 (same as model model_5 but with 50% less training data)

In [26]:
base_model = tf.keras.applications.EfficientNetB0(False)
base_model.trainable=False

inputs = tf.keras.layers.Input(shape=(200, 200, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(29)(x)
outputs = tf.keras.layers.Activation(tf.keras.activations.softmax)(x)
model_8 = tf.keras.Model(inputs, outputs)

model_8.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_8 = model_8.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=int(0.5 *len(train_data)), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_8")]
)

Epoch 1/3
1087/1087 [==============================] - 79s 67ms/step - loss: 0.6234 - accuracy: 0.8934 - val_loss: 0.1760 - val_accuracy: 0.9773
Epoch 2/3
1087/1087 [==============================] - 68s 63ms/step - loss: 0.1500 - accuracy: 0.9793 - val_loss: 0.0839 - val_accuracy: 0.9901
Epoch 3/3
   1/1087 [..............................] - ETA: 57s - loss: 0.1902 - accuracy: 1.0000

1087/1087 [==============================] - 10s 9ms/step - loss: 0.1902 - accuracy: 1.0000 - val_loss: 0.0844 - val_accuracy: 0.9899


### model_9(same as model_5 but with 25% of training data)

In [27]:
base_model = tf.keras.applications.EfficientNetB0(False)
base_model.trainable=False

inputs = tf.keras.layers.Input(shape=(200, 200, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(29)(x)
outputs = tf.keras.layers.Activation(tf.keras.activations.softmax)(x)
model_9 = tf.keras.Model(inputs, outputs)

model_9.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_9 = model_9.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=int(0.25 *len(train_data)), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_9")]
)

Epoch 1/3
543/543 [==============================] - 48s 78ms/step - loss: 0.9583 - accuracy: 0.8244 - val_loss: 0.3423 - val_accuracy: 0.9552
Epoch 2/3
543/543 [==============================] - 41s 75ms/step - loss: 0.2797 - accuracy: 0.9608 - val_loss: 0.1728 - val_accuracy: 0.9786
Epoch 3/3
543/543 [==============================] - 38s 69ms/step - loss: 0.1721 - accuracy: 0.9746 - val_loss: 0.1152 - val_accuracy: 0.9844


### model_10 (same as model_5 but with 10% of training data)

In [28]:
base_model = tf.keras.applications.EfficientNetB0(False)
base_model.trainable=False

inputs = tf.keras.layers.Input(shape=(200, 200, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(29)(x)
outputs = tf.keras.layers.Activation(tf.keras.activations.softmax)(x)
model_10 = tf.keras.Model(inputs, outputs)

model_10.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer=tf.keras.optimizers.Adam()
)
history_10 = model_10.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=int(0.10 *len(train_data)), 
    validation_data=test_data, 
    validation_steps=int(0.25 * len(test_data)), 
    callbacks=[tf.keras.callbacks.TensorBoard("tensorboard/model_10")]
)

Epoch 1/3
217/217 [==============================] - 27s 95ms/step - loss: 1.5997 - accuracy: 0.6770 - val_loss: 0.7651 - val_accuracy: 0.8902
Epoch 2/3
217/217 [==============================] - 22s 103ms/step - loss: 0.6100 - accuracy: 0.9119 - val_loss: 0.4208 - val_accuracy: 0.9416
Epoch 3/3
217/217 [==============================] - 22s 104ms/step - loss: 0.3959 - accuracy: 0.9450 - val_loss: 0.2960 - val_accuracy: 0.9589


### Comparing all the models that we have built with the test data

In [42]:
import pandas as pd

In [37]:
tf.keras.models.save_model(model_0, "/content/drive/MyDrive/Project_signlanguage/model_0")

In [38]:
tf.keras.models.save_model(model_1, "/content/drive/MyDrive/Project_signlanguage/model_1")
tf.keras.models.save_model(model_2_1, "/content/drive/MyDrive/Project_signlanguage/model_2_1")
tf.keras.models.save_model(model_2_2, "/content/drive/MyDrive/Project_signlanguage/model_2_2")
tf.keras.models.save_model(model_2_3, "/content/drive/MyDrive/Project_signlanguage/model_2_3")

In [ ]:
tf.keras.models.save_model(model_3, "/content/drive/MyDrive/Project_signlanguage/model_3")
tf.keras.models.save_model(model_3N, "/content/drive/MyDrive/Project_signlanguage/model_3N")
tf.keras.models.save_model(model_4, "/content/drive/MyDrive/Project_signlanguage/model_4")
tf.keras.models.save_model(model_5, "/content/drive/MyDrive/Project_signlanguage/model_5")

In [ ]:
tf.keras.models.save_model(model_6, "/content/drive/MyDrive/Project_signlanguage/model_6")
tf.keras.models.save_model(model_7, "/content/drive/MyDrive/Project_signlanguage/model_7")
tf.keras.models.save_model(model_8, "/content/drive/MyDrive/Project_signlanguage/model_8")
tf.keras.models.save_model(model_9, "/content/drive/MyDrive/Project_signlanguage/model_9")
tf.keras.models.save_model(model_10, "/content/drive/MyDrive/Project_signlanguage/model_10")

In [32]:
!tensorboard dev upload --logdir ./tensorboard \
  --name "sign language project experiments" \
  --description "A series of different experiment" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./tensorboard

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=okhuq4acbJkLzreb1r3fnSRp4PnTBT&prompt=consent&access_type=offline